In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
import plotly as py
import plotly.express as px

import plotly.graph_objs as go
import plotly.figure_factory as ff

# Introduction

In this lecture we examine the process of Exploratory Data Analysis (EDA).  Often you will acquire or even be given a collection of data in order to conduct some analysis or answer some questions. The first step in using that data is to ensure that it is in the correct form (cleaned) and that you understand its properties and limitations (EDA).  Often as you explore data through EDA you will identify additional transformations that may be required before the data is ready for analysis.

In this notebook we obtain crime data from the city of Berkeley's public records.  Ultimately, our goal might be to understand policing patterns but before we get there we must first clean and understand the data. 

# Getting the Data

To begin this analysis we want to get data about crimes in Berkeley.  Remarkably, the city of Berkeley maintains an [Open Data Portal](https://data.cityofberkeley.info/) for citizens to access data about the city.  We will be examining the:

1. [Call Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5)
1. [Stop Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Stop-Data-October-1-2020-Present-/ysvs-bcge)

Fortunately, this data is also relatively well document with detailed descriptions of what it contains. 

For this lecture, I am downloading a pre-extracted version.

In [ ]:
def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.
    
    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded 
    
    return: The pathlib.Path object representing the file.
    """
    import requests
    from pathlib import Path
    import time 
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok = True)
    file_path = data_dir / Path(file)
    # If the file already exists and we want to force a download then
    # delete the file first so that the creation date is correct.
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
        last_modified_time = time.ctime(file_path.stat().st_mtime)
    else:
        last_modified_time = time.ctime(file_path.stat().st_mtime)
        print("Using cached version that was downloaded (UTC):", last_modified_time)
    return file_path
    

In [ ]:
calls_file = fetch_and_cache("https://raw.githubusercontent.com/DS-100/oreilly-bootcamp/main/data/calls.csv",
                             "calls.csv")
stops_file = fetch_and_cache("https://raw.githubusercontent.com/DS-100/oreilly-bootcamp/main/data/stops.json",
                             "stops.json")


## Most data has bad documentation:

Unfortunately, data is seldom well documented and when it is you may not be able to trust the documentation. It is therefore critical that when we download the data we investigate the fields and verify that it reflects the assumptions made in the documentation.



# Exploring the data

Now that we have obtained the data we want to understand its:

* **Structure** -- the "shape" of a data file
* **Granularity** -- how fine/coarse is each datum
* **Scope** -- how (in)complete is the data
* **Temporality** -- how is the data situated in time
* **Faithfulness** -- how well does the data capture "reality"



## Structure

Before we even begin to load the data it often helps to understand a little about the high-level structure:

1. How much data do I have?
1. How is it formatted?

### How big is the data?

I often like to start my analysis by getting a rough estimate of the size of the data.  This will help inform the tools I use and how I view the data.  If it is relatively small I might use a text editor or a spreadsheet to look at the data.  If it is larger, I might jump to more programmatic exploration or even used distributed computing tools.

However here we will use python tools to probe the file.

In [ ]:
import os
print(calls_file, "is",  os.path.getsize(calls_file) / 1e6, "MB")
print(stops_file, "is", os.path.getsize(stops_file) / 1e6, "MB")

All the files are relatively small and we could comfortable examine them in a text editors.  (Personally, I like *sublime* or *emacs* but others may have a different *vi*ew.). 

In listing the files I noticed that the names suggest that they are all text file formats:
* **CSV**: Comma separated values is a very standard table format.
* **JSON**: JavaScript Object Notation is a very standard semi-structured file format used to store nested data.

We will dive into the formats in a moment.  However because these are text data I might also want to investigate the number of lines which often correspond to records.

In [ ]:
with open(calls_file, "r") as f:
    print(calls_file, "is", sum(1 for l in f), "lines.")

In [ ]:
with open(stops_file, "r") as f:
    print(stops_file, "is", sum(1 for l in f), "lines.")


### What is the file format?  (Can we trust extensions?)

We already noticed that the files end in `csv` and `json` which suggests that these are comma separated and javascript object files respectively.  However, we can't always rely on the naming as this is only a convention.  For example, here we picked the name of the file when downloading based on some hints in the URL.



**Often files will have incorrect extensions or no extension at all.**

Let's assume that these are text files (and do not contain binary encoded data) so we can print a "few lines" to get a better understanding of the file.

In [ ]:
print(calls_file, "======================")
with open(calls_file, "r") as f:
    for i in range(5):
        print(i, "\t", repr(f.readline()))

### What are some observations about `Calls` data?


<details><summary>Click for Solution</summary>

1. It appears to be in comma separated value (CSV) format.
1. First line contains the column headings.
1. There are lots of **new-line** `\n` characters:
    * at the ends of lines (delimiting records?)
    * *within records* as part of addresses.
1. There are **"quoted"** strings in the `Block_Location` column:
```
"2500 LE CONTE AVE
Berkeley, CA
(37.876965, -122.260544)"
```
these are going to be difficult.  What are the implications on our earlier line count calculations?


</details>

In [ ]:
print(stops_file, "======================")
with open(stops_file, "r") as f:
    for i in range(40):
        print(i, "\t", repr(f.readline()))

Notice that I used the `repr` function to return the raw string with special characters.  This is helpful in deducing the file format.


## Loading the Data

We will now attempt to load the data into python.  We will be using the Pandas dataframe library for basic tabular data analysis.  Fortunately, the Pandas library has some relatively sophisticated functions for loading data. 

Because the file appears to be a relatively well formatted CSV we will attempt to load it directly and allow the Pandas Library to deduce column headers.  (Always check that first row and column look correct after loading.)

In [ ]:
calls = pd.read_csv(calls_file, on_bad_lines="warn")
calls.tail(5)

In [ ]:
stops = pd.read_json(stops_file)
stops.tail(5)

Looking more into the structure:

In [ ]:
calls.shape

In [ ]:
stops.shape

In [ ]:
calls.columns

In [ ]:
stops.columns

<br><br><br><br>


---


# EDA on Calls Data

Often the first step of EDA is addressing any obvious issues with the data:


In [ ]:
calls.head()

What are some issues with this data?

<details><summary>Click for Solution</summary>

1. `EVENTDT` -- Contain the incorrect time stamp
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `CVDOW` -- Appears to be some encoding of the day of the week (see data documentation).
1. `InDbDate` -- Appears to be correctly formatted and appears pretty consistent in time.
1. **`Block_Location` -- Errr, what a mess!  newline characters, and Geocoordinates all merged!!  Fortunately, this field was "quoted" otherwise we would have had trouble parsing the file. (why?)**
1. `BLKADDR` -- This appears to be the address in Block Location.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.

</details>

<br><br><br><br>


Often I will go column by column looking for things that might be interesting.

<br><br><br>

---

## Case Numbers

These are probably the primary key but we want to verify that and also examine how they were assigned. 


### Primary Key

Case numbers are probably used internally to track individual cases and my reference other data we don't have access to.  However, it is possible that multiple calls could be associated with the same case.  Let's see if the case numbers are all unique.


In [ ]:
print("There are", calls['CASENO'].unique().shape[0], "unique case numbers.")
print("There are", calls.shape[0], "calls in the table.")

### Case Numbers and Time 

Sometimes something as simple as a case number and when it was created can reveal something about how the case was processed.

However we need to clean up the date and time encoding

In [ ]:
calls.head(2)

Merging `EVENTDT` and `EVENTTM` to construct a single date time object.

In [ ]:
def combine_date_time(dates, times):
    from datetime import datetime
    return pd.concat([dates, times], axis=1).apply(
        lambda r: datetime.combine(r[0], r[1]), axis=1)

dates = pd.to_datetime(calls['EVENTDT']).dt.date
times = pd.to_datetime(calls['EVENTTM']).dt.time
calls['date'] = combine_date_time(dates, times)

Plotting the cases numbers in time.

In [ ]:
px.scatter(calls, x="date", y="CASENO", hover_name="CVLEGEND")

I like to use interactive plotting tools so I can hover the mouse over the plot and read the values.  The cufflinks library adds plotly support to Pandas.  

**What might we be observing?**

<br><br><br>

--- 

## The Offense Field

The Offense field appears to contain the specific crime being reported.  As nominal data we might want to see a summary constructed by computing counts of each offense type:

In [ ]:
px.histogram(calls, x="OFFENSE").update_xaxes(categoryorder='total descending')

#### Observations?

<br><br><br>

---


## CVLEGEND

The CVLEGEND field provides the broad category of crime and is a good mechanism to group potentially similar crimes. 

In [ ]:
px.histogram(calls, x="CVLEGEND").update_xaxes(categoryorder='total descending')

Notice that when we group by the crime time we see that **larceny** emerges as one of the top crimes.  Larceny is essentially stealing -- taking someone else stuff without force.

### What is the relatinoship between Offense and CVLegend

In [ ]:
pvt = pd.pivot_table(calls, values="CASENO", index="OFFENSE", columns="CVLEGEND", aggfunc="count", fill_value=0)
pvt

In [ ]:
px.imshow(pvt.to_numpy().T, x=pvt.index, y=pvt.columns, height=800)

<br><br><br>

---



## Time Fields

We already started to look at where this data is situated in time and how calls were reported during the reporting period.  Now lets examine how calls relate to day of the week and even the type of crime.

In [ ]:
calls.head(2)

Again, we need to do some data cleaning.  `CVDOW` contains the day of the week.  According to the documentation `CVDOW=0` is Sunday, `CVDOW=1` is Monday, ...,  Therefore we can make a series to decode the day of the week for each record and join that series with the calls data.

In [ ]:
dow = pd.Series(["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"], name="Day")
dow

In [ ]:
calls = pd.merge(calls, pd.DataFrame(dow), left_on='CVDOW', right_index=True).sort_index()
calls.head()

In [ ]:
px.histogram(calls, "Day", category_orders={"Day": dow})

**Observations?**

<br><br><br>

### How about temporal patterns within a day?

In [ ]:
calls['hour_of_day'] = (
    calls['date'].dt.hour * 60 + calls['date'].dt.minute ) / 60.

In [ ]:
ff.create_distplot([calls['hour_of_day']], group_labels=["Hour"],bin_size=.5, show_rug=False)

**Observations?**

<details><summary>Click for Solution</summary>
    
In the above plot we see the standard pattern of limited activity early in the morning around here 6:00AM.  We also see that large spikes at 0 and 12.  Why?

</details>

<br><br><br>


### Stratified Analysis

To better understand the time of day a report occurs we could stratify the analysis by the day of the week.  To do this we will use box plots.  


In [ ]:
px.violin(calls.sort_values("CVDOW"), y="hour_of_day", x="Day", box=True, points="all", hover_name="CVLEGEND")

**Observations?**

<br><br><br>

---

## Location

The block location contains the lat/lon coordinates and I might want to use these to analyze the location of each request.  Let's try to extract the GPS coordinates using regular expressions (we will cover regular expressions in future lectures):


In [ ]:
calls['Block_Location'].head(10)

In [ ]:
calls_lat_lon = (
    # Remove newlines
    calls['Block_Location'].str.replace("\n", "\t") 
    # Extract Lat and Lon using regular expression
    .str.extract(".*\((?P<Lat>\d*\.\d*)\, (?P<Lon>-?\d*\.\d*)\)", expand=True)
    .astype(float)
)
calls_lat_lon.head(20)

The following block of code joins the extracted Latitude and Longitude fields with the calls data.  Notice that we actually drop these fields before joining.  This is to enable repeated invocation of this cell even after the join has been completed. 

In [ ]:
# Join in the the latitude and longitude data
calls = calls.join(calls_lat_lon)
# calls[["Lat", "Lon"]] = calls_lat_lon
calls.head()

In [ ]:
px.set_mapbox_access_token(open("mapbox.token").read())

In [ ]:
px.density_mapbox(calls, lat='Lat', lon='Lon', radius=7, zoom=12,
                        mapbox_style="stamen-terrain", height=800)

### Questions

1. Why are all the calls located on the street and at often at intersections?


In [ ]:
calls.head(2)

In [ ]:
px.scatter_mapbox(calls, lat="Lat", lon="Lon", color="CVLEGEND", size_max=10, zoom=12, height=800)

# You Explore

What else do you see in this data?  Try playing with it.

<br><br><br><br>


---


# EDA on Stops Data

## You try


In [ ]:
stops.head()

<br><br><br><br>


---


# EDA on Stops Data

Often the first step of EDA is addressing any obvious issues with the data:


In [ ]:
stops.head()

<br><br><br>

--- 

## Record Numbers

In [ ]:
print("Unique Records:", len(stops['lea_record_id'].unique()))
print("Total Records:", len(stops))

In [ ]:
px.histogram(stops["lea_record_id"].value_counts(), log_y=True)

<br><br><br>

### Temporality

In [ ]:
dates = pd.to_datetime(stops['date_of_stop']).dt.date
times = pd.to_datetime(stops['time_of_stop']).dt.time
stops['date'] = combine_date_time(dates, times)

In [ ]:
stops['record_num'] = stops['lea_record_id'].str.replace("BPD", "").astype(float)

In [ ]:
px.scatter(stops, x="date", y="record_num")

<br><br><br>

---

## Person Number

In [ ]:
px.histogram(stops, x="person_number")

<br><br><br>

---

## Perceived Race

In [ ]:
px.histogram(stops, x='perceived_race_or_ethnicity') # log_y=True)

In [ ]:
px.histogram(stops[~stops["perceived_race_or_ethnicity"].str.contains("\|")], x='perceived_race_or_ethnicity') # log_y=True)

Relationship to whether the race was perceived prior to the stop:

In [ ]:
px.histogram(stops[~stops["perceived_race_or_ethnicity"].str.contains("\|")], 
             x='perceived_race_or_ethnicity', color="raceperceivedpriortostop", barmode="group")

<br><br><br>

---

## City of Residence

In [ ]:
px.histogram(stops, x="city_of_residence")

In [ ]:
top_cities = (stops
              .groupby("city_of_residence", as_index=False).size()
              .sort_values("size", ascending=False)
              .head(20))
px.bar(top_cities, x="city_of_residence", y="size")

<br><br><br>

---

##  Understanding Age and Gender

In [ ]:
px.histogram(stops, nbins=20, 
             x="perceived_age", color="perceived_gender",  
             barmode="overlay", histnorm="percent")